In [1]:
import pandas as pd
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from datetime import datetime
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt

In [2]:
# loading the data
df = pd.read_csv("loan.csv")
df.shape
df_raw = df.copy()

C:\Users\tamar\AppData\Local\Temp\ipykernel_19020\961585608.py:2: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("loan.csv")


In [3]:
# -------------------------------------------------------------
# Step 1: Create the 3-class target (common for both pipelines)
# -------------------------------------------------------------
def create_target(df):
    """
    Creates the 3-class target:
      - paid_on_time
      - paid_late
      - not_paid

    Also computes helper columns needed to classify paid_late correctly.
    """
    df = df.copy()

    df['issue_d'] = pd.to_datetime(df['issue_d'], format='%b-%Y', errors='coerce')
    df['last_pymnt_d'] = pd.to_datetime(df['last_pymnt_d'], format='%b-%Y', errors='coerce')

    # Extract loan term in months
    df['term_months'] = df['term'].astype(str).str.extract(r'(\d+)').astype(float)

    # Approximate expected end date
    df['expected_end'] = df['issue_d'] + pd.to_timedelta(df['term_months'] * 30, unit='D')

    # Late fully paid flag
    df['paid_late_flag'] = (
        (df['loan_status'] == 'Fully Paid') &
        (df['last_pymnt_d'] > df['expected_end'])
    )

    # Build target variable
    df['target_3class'] = 'paid_on_time'
    df.loc[df['paid_late_flag'], 'target_3class'] = 'paid_late'
    df.loc[df['loan_status'].isin(['Charged Off', 'Default']), 'target_3class'] = 'not_paid'
    # Remove '(future leakage + breaks categorical encoding)
    df = df.drop(columns=['next_pymnt_d', 'paid_late_flag', 'last_pymnt_d'], errors='ignore')


    return df[df['target_3class'].notna()].reset_index(drop=True)


In [4]:
# =============================================================
# FULL CLEAN PIPELINE (30 variables clean)
# =============================================================
leakage_columns = [
    'hardship_flag','debt_settlement_flag','total_pymnt','total_rec_prncp',
    'total_rec_int','total_rec_late_fee','last_pymnt_d','last_pymnt_amnt',
    'recoveries','collection_recovery_fee','out_prncp','total_pymnt_inv',
    'out_prncp_inv','loan_status'
]

high_corr_drop_columns = [
    'funded_amnt','funded_amnt_inv','installment',
    'num_rev_tl_bal_gt_0','tot_hi_cred_lim',
    'total_il_high_credit_limit','num_sats'
]

redundant_columns = ['policy_code','disbursement_method','chargeoff_within_12_mths', 'initial_list_status']


def basic_clean1(df):
    """
    First stage of full clean:
    - Remove leakage, high correlation, and redundant columns
    - Drop columns with >90% missing
    - Clean term and emp_length
    - Remove rows with missing target
    """
    clean_df = df.copy()

    cols_to_drop = [c for c in (leakage_columns + high_corr_drop_columns + redundant_columns) 
                    if c in clean_df.columns]
    clean_df = clean_df.drop(columns=cols_to_drop)

    # Drop columns with excessive missingness
    missing_pct = clean_df.isnull().mean() * 100
    high_missing_cols = missing_pct[missing_pct > 90].index.tolist()
    clean_df = clean_df.drop(columns=high_missing_cols)

    # Remove "Not Verified"
    clean_df = clean_df[clean_df['verification_status'] != 'Not Verified']
    clean_df = clean_df.drop(columns=['verification_status'], errors='ignore')

    # Clean term (36/60)
    clean_df['term'] = clean_df['term'].astype(str).str.extract(r'(\d+)').astype(float)

    # Clean employee length
    emp = clean_df['emp_length'].astype(str)
    emp = emp.str.replace('< 1', '0', regex=False)
    clean_df['emp_length'] = emp.str.extract(r'(\d+)').astype(float)

    return clean_df[clean_df['target_3class'].notna()].reset_index(drop=True)


def basic_clean2(df):
    """
    Second stage of full clean:
    - Remove non-informative columns
    - Normalize text columns
    - Create engineered features (purpose_grouped, home_stability, credit_age_years, etc.)
    - Remove raw columns after feature engineering
    """
    clean_df = df.copy()

    # Non-informative
    for col in ['pymnt_plan','zip_code','collections_12_mths_ex_med']:
        clean_df = clean_df.drop(columns=col, errors='ignore')

    # Lowercase text columns
    for col in ['purpose','home_ownership','addr_state','application_type','emp_title']:
        if col in clean_df:
            clean_df[col] = clean_df[col].astype(str).str.lower().str.strip()

    # Grouped purpose variable
    clean_df['purpose_grouped'] = clean_df['purpose'].replace({
        'debt_consolidation':'debt','credit_card':'debt',
        'home_improvement':'housing','house':'housing',
        'small_business':'business',
        'car':'personal','medical':'personal','vacation':'personal','moving':'personal',
        'wedding':'personal','major_purchase':'personal',
        'renewable_energy':'other','educational':'other','other':'other'
    })

    # Home stability
    clean_df['home_stability'] = clean_df['home_ownership'].replace({
        'mortgage':'stable','own':'stable',
        'rent':'unstable','none':'unstable','other':'unstable'
    })

    # Credit age
    years = clean_df['earliest_cr_line'].astype(str).str.extract(r'(\d{4})')[0].astype(float)
    clean_df['credit_age_years'] = datetime.now().year - years

    # Create engineered count-like fields
    clean_df['bad_records_count'] = clean_df[['pub_rec','pub_rec_bankruptcies','tax_liens']].sum(axis=1, min_count=1)
    clean_df['recent_credit_activity'] = (
        clean_df['inq_last_6mths'] + clean_df['num_tl_op_past_12m'] - (clean_df['mths_since_recent_inq']/12)
    )

    clean_df['total_balance_all'] = clean_df['tot_cur_bal'] + clean_df['total_bal_il']
    clean_df['active_credit_accounts'] = clean_df['num_actv_bc_tl'] + clean_df['num_actv_rev_tl']

    # Binary delinquency flag
    clean_df['any_delinquency'] = (
        (clean_df['num_accts_ever_120_pd']>0) |
        (clean_df['num_tl_120dpd_2m']>0) |
        (clean_df['num_tl_90g_dpd_24m']>0) |
        (clean_df['num_tl_30dpd']>0) |
        (clean_df['delinq_2yrs']>0)
    ).astype(int)

    clean_df['is_joint_app'] = clean_df['application_type'].str.contains('joint').astype(int)
    clean_df['has_current_delinquency'] = (clean_df['acc_now_delinq']>0).astype(int)
    clean_df['has_collections'] = (clean_df['tot_coll_amt']>0).astype(int)

    # Drop raw columns after creating engineered features
    columns_to_remove = [
        'purpose','home_ownership','earliest_cr_line','application_type',
        'acc_now_delinq','tot_coll_amt','title','emp_title',
        'addr_state','issue_d','last_credit_pull_d','loan_status',
        'days_late','open_acc','revol_bal','pub_rec','pub_rec_bankruptcies',
        'tax_liens','inq_last_6mths','num_tl_op_past_12m','mths_since_recent_inq',
        'tot_cur_bal','total_bal_il','num_actv_bc_tl','num_actv_rev_tl',
        'num_accts_ever_120_pd','num_tl_120dpd_2m','num_tl_90g_dpd_24m',
        'delinq_2yrs','num_tl_30dpd'
    ]

    clean_df = clean_df.drop(columns=[c for c in columns_to_remove if c in clean_df.columns])

    return clean_df



def basic_clean3(df):
    """
    Final stage of full clean:
    Removes noisy, sparse, redundant, or low-importance features.
    """
    clean_df = df.copy()

    low_importance = [
        'open_act_il','open_il_12m','open_il_24m','open_rv_12m','open_rv_24m',
        'open_acc_6m','inq_fi','total_cu_tl','acc_open_past_24mths'
    ]

    redundant = [
        'num_bc_sats','num_bc_tl','num_il_tl','num_op_rev_tl',
        'num_rev_accts','total_acc'
    ]

    sparse = [
        'mths_since_last_record','mths_since_recent_bc_dlq',
        'mths_since_recent_revol_delinq','percent_bc_gt_75'
    ]

    noisy = [
        'mo_sin_old_il_acct','mo_sin_old_rev_tl_op',
        'mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl',
        'mths_since_rcnt_il'
    ]

    optional = [
        'mths_since_last_delinq','avg_cur_bal','max_bal_bc','all_util','il_util',
        'inq_last_12m','pct_tl_nvr_dlq','mort_acc','total_bc_limit','total_acc'
    ]

    to_drop = low_importance + redundant + sparse + noisy + optional
    clean_df = clean_df.drop(columns=[c for c in to_drop if c in clean_df.columns])

    return clean_df



def full_clean(df):
    """
    Applies the full 3-stage cleaning process:
    basic_clean1 → basic_clean2 → basic_clean3
    """
    df1 = basic_clean1(df)
    df2 = basic_clean2(df1)
    df3 = basic_clean3(df2)
    return df3

In [5]:
# =============================================================
# MINIMAL CLEAN PIPELINE
# =============================================================
def minimal_clean1(df):
    """
    Minimal clean:
    - Remove leakage
    - Remove columns with >90% missing
    - Remove one feature from each high-correlation (>0.95) pair
    - Extreme-value capping
    """
    clean_df = df.copy()

    clean_df.drop(columns = ["pymnt_plan", "policy_code"])  #fixed values columns

    # deleting non-verified data
    clean_df = clean_df.drop(clean_df[clean_df["verification_status"] == "Not Verified"].index)

    leakage = [
        'hardship_flag','debt_settlement_flag','total_pymnt','total_rec_prncp',
        'total_rec_int','total_rec_late_fee','last_pymnt_d','last_pymnt_amnt',
        'recoveries','collection_recovery_fee','out_prncp','total_pymnt_inv',
        'out_prncp_inv','loan_status','paid_late_flag'
    ]
    clean_df = clean_df.drop(columns=[c for c in leakage if c in clean_df.columns])

    # Drop >90% missing
    missing_pct = clean_df.isnull().mean() * 100
    high_missing = missing_pct[missing_pct > 90].index.tolist()
    clean_df = clean_df.drop(columns=high_missing)


    # High-correlation removal
    num = clean_df.select_dtypes(include='number').columns
    if len(num) > 1:
        corr = clean_df[num].corr().abs()
        to_drop = set()

        missing = clean_df[num].isnull().mean()
        var = clean_df[num].var()

        for i, c1 in enumerate(num):
            for j, c2 in enumerate(num):
                if j <= i:
                    continue
                if corr.loc[c1, c2] > 0.95:
                    if missing[c1] > missing[c2]:
                        to_drop.add(c1)
                    elif missing[c2] > missing[c1]:
                        to_drop.add(c2)
                    else:
                        to_drop.add(c1 if var[c1] < var[c2] else c2)

        clean_df = clean_df.drop(columns=list(to_drop))




    # Extreme-value capping
    cols_to_cap_extreme = [
    "annual_inc",
    "revol_bal",
    "open_il_12m",
    "mo_sin_old_il_acct",
    "dti",
    "mo_sin_old_rev_tl_op"
]

    for column in cols_to_cap_extreme:
        cap_value = clean_df[column].quantile(0.99999)
        clean_df.loc[clean_df[column] > cap_value, column] = cap_value


    return clean_df



def minimal_clean2(df):
    """
    Additional minimal cleaning:
    Convert term and emp_length to numeric, extract issue year.
    """
    clean_df = df.copy()

    clean_df['term'] = clean_df['term'].astype(str).str.extract(r'(\d+)').astype(float)

    emp = clean_df['emp_length'].astype(str)
    emp = emp.str.replace('< 1', '0', regex=False)
    clean_df['emp_length'] = emp.str.extract(r'(\d+)').astype(float)

    clean_df['issue_year'] = clean_df['issue_d'].astype(str).str.extract(r'(\d{4})')[0].astype(float)

    return clean_df



def minimal_clean(df):
    """
    Applies minimal_clean1 → minimal_clean2
    """
    df1 = minimal_clean1(df)
    df2 = minimal_clean2(df1)
    return df2[df2["target_3class"].notna()].reset_index(drop=True)

In [6]:
def make_preprocess(df):
    """
    Builds preprocessing:
      - Time since event columns (mths_since_*):
          * Impute missing with 0 (interpreted as 'no event')
          * Add missing indicator
          * Scale
      - Other numeric columns:
          * Impute missing with median
          * Add missing indicator
          * Scale
      - Categorical columns:
          * Impute missing with string 'missing'
          * One-hot encode (missing becomes category)
    """

    # 1. Identify column groups
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    time_cols = [col for col in numeric_cols if col.startswith("mths_since_")]
    num_regular = list(set(numeric_cols) - set(time_cols))
    categorical_cols = df.select_dtypes(include=['object', 'bool']).columns.tolist()

    # 2. Pipelines
    time_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="constant", fill_value=0, add_indicator=True)),
        ("scaler", StandardScaler())
    ])

    numeric_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median", add_indicator=True)),
        ("scaler", StandardScaler())
    ])

    categorical_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ])

    # 3. Combine all
    transformer = ColumnTransformer(
        transformers=[
            ("time", time_pipeline, time_cols),
            ("num", numeric_pipeline, num_regular),
            ("cat", categorical_pipeline, categorical_cols),
        ],
        remainder="drop"
    )

    return transformer

In [7]:
def build_model_pipeline(preprocess):
    """
    Combines preprocessing + RandomForest classifier into a single Pipeline.
    """
    model = RandomForestClassifier(n_estimators=300, random_state=42)

    pipe = Pipeline([
        ("preprocess", preprocess),
        ("clf", model)
    ])

    return pipe

In [ ]:
def run_experiment(df, title, model_type):
    """
    Runs training + test split + preprocessing + model training.
    model_type: "random_forest", "logistic", "xgboost"
    """
    print(f"\n=== Running: {title} ({model_type}) ===")

    # Split into features and target
    X = df.drop(columns=["target_3class"])
    y = df["target_3class"]
    
        # Convert target to numeric for XGBoost
    if model_type == "xgboost":
        y = y.astype("category").cat.codes

    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2,
        stratify=y,
        random_state=42,
    )


    # Build preprocessing based only on the feature matrix
    preprocess = make_preprocess(X_train)

    # Choose model
    if model_type == "random_forest":
        model = RandomForestClassifier(
            n_estimators=650,
            max_depth=None,
            min_samples_split=7,
            min_samples_leaf=1,
            max_features=0.5,
            n_jobs=-1,
            random_state=42
        )

    elif model_type == "xgboost":
        model = XGBClassifier(
            n_estimators=550,
            learning_rate=0.01,
            max_depth=8,
            subsample=0.7,
            colsample_bytree=0.7,
            min_child_weight=1,
            gamma=0.6,
            eval_metric="logloss",
            tree_method="hist"   # FAST + GPU compatible
        )

    elif model_type == "logistic":
        model = LogisticRegression(
            penalty='l2',
            max_iter=2000,
            l1_ratio=0.8,
            class_weight="balanced",
            C=0.05
        )

    # Create pipeline
    pipe = Pipeline([
        ("pre", preprocess),
        ("model", model)
    ])

    # Fit
    pipe.fit(X_train, y_train)

    # ---------- Evaluation ----------
    # תחזיות על קבוצת ה-test
    y_pred = pipe.predict(X_test)

    # 1. Accuracy – אחוז הדוגמאות שנחזו נכון
    acc = accuracy_score(y_test, y_pred)

    # 2. Macro F1 – ממוצע F1 לכל המחלקות
    macro_f1 = f1_score(y_test, y_pred, average="macro")

    print("Accuracy:", acc)
    print("Macro F1:", macro_f1)

    # 3. Confusion Matrix
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    # 4. Classification Report – Precision / Recall / F1 לכל מחלקה
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    return acc, pipe

In [ ]:
# =============================================================
# Run 3 models on both Full and Minimal datasets
# =============================================================
df_with_target = create_target(df_raw)

df_sample = df_with_target.sample(100000, random_state=42)

df_full_sample = full_clean(df_sample.copy())
df_minimal_sample = minimal_clean(df_sample.copy())


models = ["random_forest", "xgboost", "logistic"]

results = {}

for model_name in models:
    print("\n=============================================================")
    print(f"MODEL: {model_name.upper()}")
    print("=============================================================")

    # ---- FULL CLEAN ----
    acc_full, model_full = run_experiment(
        df_full_sample,
        f"Full Clean Sample - {model_name}",
        model_type=model_name
    )

    # ---- MINIMAL CLEAN ----
    acc_minimal, model_minimal = run_experiment(
        df_minimal_sample,
        f"Minimal Clean Sample - {model_name}",
        model_type=model_name
    )

    # Save results
    results[(model_name, "full")] = acc_full
    results[(model_name, "minimal")] = acc_minimal


# =============================================================
# Print Summary Table
# =============================================================
print("\n\n==================== SUMMARY ====================\n")
print("{:<20} {:<15} {:<15}".format("Model", "Full Clean", "Minimal Clean"))
print("-" * 50)

for model_name in models:
    print("{:<20} {:<15.4f} {:<15.4f}".format(
        model_name,
        results[(model_name, "full")],
        results[(model_name, "minimal")]
    ))

### handling outliers


In [ ]:
full_df = create_target(df_raw)
minimal_clean_full_df = minimal_clean(full_df)

In [ ]:
for col in minimal_clean_full_df.columns:
    plt.figure(figsize=(8, 4))
    plt.title(col)

    # Numeric columns
    if pd.api.types.is_numeric_dtype(minimal_clean_full_df[col]):
        sns.histplot(minimal_clean_full_df[col].dropna(), kde=True)
        plt.xlabel(col)

        # Categorical or text columns
    else:
        num_unique = minimal_clean_full_df[col].nunique()

        if num_unique <= 15:
            minimal_clean_full_df[col].value_counts().plot(kind="bar")
            plt.xlabel(col)
            plt.ylabel("Count")
        else:
            print(f"Skipping '{col}' (too many categories: {num_unique})")
            plt.close()
            continue

    plt.tight_layout()
    plt.show()

In [ ]:
# Create the boxplot
plt.figure(figsize=(10, 4))
sns.boxplot(x=minimal_clean_full_df['annual_inc'])

In [ ]:
# Create the boxplot
plt.figure(figsize=(10, 4))
sns.boxplot(x=minimal_clean_full_df['dti'])

In [ ]:
# לראות כמה ערכים יש עם DTI מעל 100%
minimal_clean_full_df[minimal_clean_full_df['dti'] > 100].shape[0]

In [ ]:
# Create the boxplot
plt.figure(figsize=(10, 4))
sns.boxplot(x=minimal_clean_full_df['open_acc'])

In [ ]:
# Create the boxplot
plt.figure(figsize=(10, 4))
sns.boxplot(x=minimal_clean_full_df['pub_rec'])


plt.show()

In [ ]:
plt.figure(figsize=(10, 4))
sns.boxplot(x=minimal_clean_full_df['revol_util'])
plt.show()

# לראות כמה ערכים יש עם  מעל 100%
minimal_clean_full_df[minimal_clean_full_df['revol_util'] > 100].shape[0]

In [ ]:
# Create the boxplot
plt.figure(figsize=(10, 4))
sns.boxplot(x=minimal_clean_full_df['open_il_12m'])

In [ ]:
plt.figure(figsize=(10, 4))
sns.boxplot(x=minimal_clean_full_df['mo_sin_old_il_acct'])

### parameter tuning

In [9]:
#checking the number of values from each column:
df_with_target = create_target(df_raw)
df_with_target["target_3class"].value_counts()

target_3class
paid_on_time    1788541
not_paid         261686
paid_late        210441
Name: count, dtype: int64

--- random forest ---

In [8]:
# 1. Load data
df_tuning = df.copy()

# 2. Create target
df_with_target_tuning = create_target(df_tuning)

# 3. Choose minimal clean
df_clean_tuning = minimal_clean(df_with_target_tuning)

# 4. Make balanced N-row dataset
def make_balanced_sample(df, N, target_col="target_3class", random_state=42):
    k = df[target_col].nunique()
    per_class = N // k

    balanced = (
        df.groupby(target_col)
          .apply(lambda g: g.sample(per_class, random_state=random_state))
          .reset_index(drop=True)
    )

    remaining = N - len(balanced)
    if remaining > 0:
        extra = df.sample(remaining, random_state=random_state)
        balanced = pd.concat([balanced, extra], ignore_index=True)

    return balanced

df_balanced_tuning = make_balanced_sample(df_clean_tuning, 20000)

# 5. Split into X and y
X = df_balanced_tuning.drop(columns=["target_3class"])
y = df_balanced_tuning["target_3class"]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

# 6. Build preprocessing on X_train only
preprocess_tuning = make_preprocess(X_train)

C:\Users\tamar\AppData\Local\Temp\ipykernel_19020\308865155.py:71: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1233073.8432078396' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  clean_df.loc[clean_df[column] > cap_value, column] = cap_value
C:\Users\tamar\AppData\Local\Temp\ipykernel_19020\132350198.py:17: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(per_class, random_state=random_state))


In [12]:
param_grid = {
    "clf__n_estimators": [200, 700, 2000]
    #1000 stops it completly
   # "model__max_depth": [None, 10, 20],
   # "model__min_samples_leaf": [1, 2, 5, 10],
   # "model__max_features": ["sqrt", 0.3, 0.5],
}

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

rf_pipe = Pipeline([
    ("preprocess", preprocess_tuning),
    ("clf", RandomForestClassifier(random_state=42, n_jobs=1))
])

In [ ]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(
    estimator=rf_pipe,
    param_grid=param_grid,
    scoring="f1_macro",
    cv=3,
    n_jobs=-1,
    verbose=2,
    refit=True
)

gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


In [27]:


cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

rf_pipe = Pipeline([
    ("preprocess", preprocess_tuning),
    ("clf", RandomForestClassifier(random_state=42, n_jobs=1))
])

rf_param_grid = {
    "clf__n_estimators": [500],
    #"clf__max_depth": [None, 10, 20, 30],
    #"clf__max_features": ["sqrt", 0.5],
    #"clf__min_samples_leaf": [1, 5, 20],
    #"clf__class_weight": [None, "balanced"],
}

rf_grid = GridSearchCV(
    rf_pipe,
    param_grid=rf_param_grid,
    scoring="f1_macro",
    cv=cv,
    n_jobs=-1,
    verbose=2
)

rf_grid.fit(X_train, y_train)
print("RF best:", rf_grid.best_score_)
print("RF params:", rf_grid.best_params_)


Fitting 3 folds for each of 1 candidates, totalling 3 fits


KeyboardInterrupt: 

In [11]:
rf_pipe_tuning = Pipeline([
    ("pre", preprocess_tuning),
    ("clf", RandomForestClassifier(
        random_state=42,
        n_jobs=-1,
    ))
])

rf_pipe_tuning.fit(X_train, y_train)
y_pred = rf_pipe_tuning.predict(X_test)

baseline_f1 = f1_score(y_test, y_pred, average="macro")
print("Baseline Macro F1:", baseline_f1)

Baseline Macro F1: 0.6790028016193377


In [21]:
rf_params_coarse = {
    # We tune in phases so the computer can handle it. After deciding on each phase, moving on to the next one.

    # first:
    #"clf__max_depth": [None, 3, 5, 8, 12, 16, 24, 32, 40],
    "clf__max_depth": [None, 24]
    #"clf__min_samples_leaf": [1, 16],
    #"clf__n_estimators": [100, 600],

    # second
    #"clf__max_features": ["sqrt", "log2", 0.7, 1.0],
    #"clf__max_features": [0.7],


    #third
    #"clf__min_samples_split": [2, 50],
    #"clf__min_samples_split": [20],
    #"clf__class_weight": [None, "balanced"]

}

rf_param_dist = {
    "n_estimators": [300, 600, 1000, 1500, 2000],
    "max_depth": [None, 3, 5, 8, 12, 16, 24, 32, 40],
    "max_features": ["sqrt", "log2", 0.2, 0.4, 0.6, 0.8, 1.0],
    "min_samples_split": [2, 5, 10, 20, 30, 50],
    "min_samples_leaf": [1, 2, 4, 8, 16, 32, 50],
    "bootstrap": [True, False],
    "max_samples": [0.5, 0.7, 0.9, 1.0],
    "class_weight": [None, "balanced", "balanced_subsample"],
}


fixed = {
    'clf__class_weight' : ['balanced'],
    "clf__max_features": [0.7]
}

In [23]:
rf_pipe = Pipeline([
    ("pre", preprocess_tuning),
    ("clf", RandomForestClassifier(max_features=0.7, class_weight = 'balanced', random_state=42, n_jobs=-1))
])

rf_search_coarse = RandomizedSearchCV(
    estimator=rf_pipe,
    param_distributions=rf_params_coarse,
    n_iter=1,
    scoring="f1_macro",
    cv=3,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

rf_search_coarse.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


KeyboardInterrupt: 

In [ ]:
print("Best coarse params:")
print(rf_search_coarse.best_params_)

print("Best CV f1_macro:")
print(rf_search_coarse.best_score_)

--- xg boost ---

In [ ]:
y_train = y_train.astype("category").cat.codes
y_test = y_test.astype("category").cat.codes

In [ ]:
xgb_pipe_tuning = Pipeline([
    ("pre", preprocess_tuning),
    ("clf", XGBClassifier(
        objective="multi:softprob",
        num_class=y.nunique(),
        eval_metric="mlogloss",
        tree_method="hist",
        random_state=42,
        n_jobs=-1
    ))
])

xgb_pipe_tuning.fit(X_train, y_train)
y_pred = xgb_pipe_tuning.predict(X_test)
baseline_xgb_f1 = f1_score(y_test, y_pred, average="macro")

print("Baseline XGB Macro F1:", baseline_xgb_f1)

In [ ]:
xgb_params_coarse = {
# We tune in phases so the computer can handle it. After deciding on each phase, moving on to the next one.

    #first
    "clf__n_estimators":        [200, 400, 800, 1200],
    "clf__learning_rate":       [0.01, 0.05, 0.1],
    "clf__max_depth":           [3, 7, 10, None],

    #second
    "clf__min_child_weight":    [1, 3, 5],
    "clf__subsample":           [0.6, 0.8, 1.0],
    "clf__colsample_bytree":    [0.6, 0.8, 1.0],
}

In [ ]:
xgb_search_coarse = RandomizedSearchCV(
    estimator=xgb_pipe_tuning,
    param_distributions=xgb_params_coarse,
    n_iter=40,
    scoring="f1_macro",
    cv=3,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

xgb_search_coarse.fit(X_train, y_train)

print("Best coarse params (XGB):")
print(xgb_search_coarse.best_params_)
print("Best CV f1_macro (XGB):")
print(xgb_search_coarse.best_score_)

--- logistic regression ---

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_pipe_tuning = Pipeline([
    ("pre", preprocess_tuning),
    ("clf", LogisticRegression(
        max_iter=2000,
        multi_class="multinomial",
        solver="lbfgs",
        n_jobs=-1
    ))
])

lr_pipe_tuning.fit(X_train, y_train)
y_pred = lr_pipe_tuning.predict(X_test)
baseline_lr_f1 = f1_score(y_test, y_pred, average="macro")

print("Baseline LR Macro F1:", baseline_lr_f1)

In [ ]:
lr_params_coarse = {
# We tune in phases so the computer can handle it. After deciding on each phase, moving on to the next one.

    #first
    "clf__C":             [1e-3, 1e-2, 1e-1, 1, 5, 10],
    "clf__solver":       ["lbfgs", "saga"],
    "clf__penalty":      ["l2", "elasticnet"],

    #second
    "clf__l1_ratio":     [0.1, 0.3, 0.5, 0.7, 0.9],
    "clf__class_weight": [None, "balanced"],
    "clf__max_iter":     [1000, 2000, 4000],
}

In [ ]:
lr_search_coarse = RandomizedSearchCV(
    estimator=lr_pipe_tuning,
    param_distributions=lr_params_coarse,
    n_iter=40,
    scoring="f1_macro",
    cv=3,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

lr_search_coarse.fit(X_train, y_train)

print("Best coarse params (LR):")
print(lr_search_coarse.best_params_)
print("Best CV f1_macro (LR):")
print(lr_search_coarse.best_score_)

In [ ]:
df_full_sample["target_3class"].value_counts()
df_full_sample["target_3class"].value_counts(normalize=True)

In [ ]:
df_minimal_sample["target_3class"].value_counts()
df_minimal_sample["target_3class"].value_counts(normalize=True)

In [ ]:
# Copy/paste standard NN (MLPClassifier) pipeline that AUTO-detects numeric vs categorical columns.
# You only need to set: target_col and df_clean.

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, roc_auc_score


# -----------------------------
# 0) INPUTS YOU SET
# -----------------------------


# 1. Load data
df_tuning = df.copy()

# 2. Create target
df_with_target_tuning = create_target(df_tuning)

# 3. Choose minimal clean
df_clean_tuning = minimal_clean(df_with_target_tuning)


df_model = df_clean_tuning.copy()       # <-- your cleaned dataframe
target_col = "target_3class"     # <-- your target column


# -----------------------------
# 1) SPLIT X / y
# -----------------------------
X = df_model.drop(columns=[target_col])
y = df_model[target_col]


# -----------------------------
# 2) AUTO-DETECT COLUMN TYPES
# -----------------------------
# Numeric: numbers + booleans
numeric_features = X.select_dtypes(include=[np.number, "bool"]).columns.tolist()

# Datetime: we'll DROP by default (you can engineer features later if needed)
datetime_features = X.select_dtypes(include=["datetime64[ns]", "datetime64[ns, UTC]"]).columns.tolist()

# Categorical: objects + pandas 'category' (excluding datetime)
categorical_features = X.select_dtypes(include=["object", "category"]).columns.tolist()

# Safety: remove datetime from X so it doesn't cause transformer issues
if datetime_features:
    X = X.drop(columns=datetime_features)


# -----------------------------
# 3) TRAIN/TEST SPLIT
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y if y.nunique() > 1 else None
)


# -----------------------------
# 4) PREPROCESSING
# -----------------------------
numeric_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, numeric_features),
        ("cat", categorical_pipe, categorical_features),
    ],
    remainder="drop",
    sparse_threshold=0.3
)


# -----------------------------
# 5) MODEL: BASIC NEURAL NET
# -----------------------------
pipe_nn = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", MLPClassifier(
        hidden_layer_sizes=(64, 32),
        activation="relu",
        solver="adam",
        alpha=1e-4,
        learning_rate_init=1e-3,
        batch_size=256,
        max_iter=50,
        early_stopping=True,
        validation_fraction=0.1,
        n_iter_no_change=10,
        random_state=42
    ))
])


# -----------------------------
# 6) FIT + EVALUATE
# -----------------------------
pipe_nn.fit(X_train, y_train)

pred = pipe_nn.predict(X_test)
print(classification_report(y_test, pred))

if hasattr(pipe_nn, "predict_proba"):
    proba = pipe_nn.predict_proba(X_test)
    if y.nunique() == 2:
        auc = roc_auc_score(y_test, proba[:, 1])
        print(f"ROC-AUC: {auc:.4f}")
    else:
        auc_ovr = roc_auc_score(y_test, proba, multi_class="ovr")
        print(f"ROC-AUC (OVR): {auc_ovr:.4f}")



# Optional: see what it auto-detected
print(f"Detected numeric cols: {len(numeric_features)}")
print(f"Detected categorical cols: {len(categorical_features)}")
print(f"Dropped datetime cols: {len(datetime_features)} -> {datetime_features[:5]}")

C:\Users\tamar\AppData\Local\Temp\ipykernel_19020\308865155.py:71: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1233073.8432078396' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  clean_df.loc[clean_df[column] > cap_value, column] = cap_value
